In [2]:
import os
import pymysql
import pandas as pd
import numpy as np
import re
from pandas.io import sql
import MySQLdb
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [3]:
conn = MySQLdb.connect(
    host="localhost",
    port=int(3306),
    user="pythonlink",
    passwd="pythonlinkp",
    db="stock_forecast_v1",
    charset='utf8mb4'
)

data = pd.read_sql_query("SELECT * FROM merged_statements500_withprice",
    conn)
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,A,2010-10-01,9696.0,6460.0,3236.0,347.6,9.33,1278.0,3.69,3691.0,...,452.5,684,718,-1174.0,601,170,-1047.5,-987.5,1.72,24.89
1,A,2011-10-01,9057.0,4741.0,4316.0,348.1,12.44,2312.0,6.67,2185.0,...,641.3,1012.0,1260.0,1294.0,-1693.0,878,2236.6,2290.4,3.09,26.51
2,A,2012-10-01,10536.0,5351.0,5185.0,347.9,14.96,1071.0,3.09,2362.0,...,668.8,1153.0,1228.0,-2366.0,-37.0,-1176.0,644.6,707.8,2.97,25.74
3,A,2013-10-01,10686.0,5397.0,5289.0,331.8,15.91,1323.0,3.98,2699.0,...,183.1,734,1152.0,-248.0,-554.0,324,320.4,387.3,2.81,36.30
4,A,2014-10-01,10815.0,5511.0,5304.0,335.3,15.83,2145.0,6.40,1663.0,...,198.8,549,731,-230.0,-117.0,353,-461.9,-393.1,1.58,39.53
5,A,2015-10-01,7479.0,3309.0,4170.0,332.2,12.57,1356.0,4.09,1655.0,...,301.9,401,1322.0,-400.0,-1089.0,-215.0,1388.9,1430.1,3.68,37.76
6,A,2016-10-01,7794.0,3548.0,4246.0,321.7,13.12,1310.0,4.05,1904.0,...,403.1,462,793,-238.0,-268.0,286,864.3,909.3,2.01,43.57
7,A,2017-10-01,8426.0,3591.0,4835.0,323.0,15.00,1863.0,5.79,2011.0,...,509.4,684,889,-304.0,-202.0,391,543.8,593.1,2.21,68.03
8,A,2018-10-01,8541.0,3970.0,4571.0,318.5,14.37,1103.0,3.47,1799.0,...,612.5,316,1087.0,-705.0,-797.0,-432.0,688.3,735.1,2.83,64.79
9,A,2019-10-01,9452.0,4704.0,4748.0,310.2,15.36,48,0.16,2407.0,...,635.6,1071.0,1021.0,-1590.0,-299.0,-866.0,846.6,892.9,2.75,75.75


In [4]:
data = data.fillna(0)

In [5]:
# Replacing cell entries with '-' with just 0
for column in data.columns:
    print("Column:", column)
    x = data.loc[data[column] == '-']
    print(x)
    if len(x) != 0:
        for element in x.index.tolist():
            data.at[element,column]= 0

Column: Company
Empty DataFrame
Columns: [Company, Time, Total Assets, Total Liabilities, Total Equity, Total Shares Out. on Filing Date, Book Value / Share, Tangible Book Value, Tangible Book Value Per Share, Total Debt, Net Debt, Total Revenues, Operating Income, Revenue Per Share, Basic EPS, Normalized Basic EPS, Dividend Per Share, EBITA, EBIT, Normalized Net Income, Net Income, Cash from Operations, Cash from Investing, Cash from Financing, Net Change in Cash, Levered Free Cash Flow, Unlevered Free Cash Flow, Free Cash Flow / Share, Price]
Index: []

[0 rows x 29 columns]
Column: Time
Empty DataFrame
Columns: [Company, Time, Total Assets, Total Liabilities, Total Equity, Total Shares Out. on Filing Date, Book Value / Share, Tangible Book Value, Tangible Book Value Per Share, Total Debt, Net Debt, Total Revenues, Operating Income, Revenue Per Share, Basic EPS, Normalized Basic EPS, Dividend Per Share, EBITA, EBIT, Normalized Net Income, Net Income, Cash from Operations, Cash from I

C:\Users\conor\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [6]:
# Replacing cell entries with 'None' with just 0
for column in data.columns:
    print("Column:", column)
    x = data.loc[data[column] == 'None']
    print(x)
    if len(x) != 0:
        for element in x.index.tolist():
            data.at[element,column]= 0

Column: Company
Empty DataFrame
Columns: [Company, Time, Total Assets, Total Liabilities, Total Equity, Total Shares Out. on Filing Date, Book Value / Share, Tangible Book Value, Tangible Book Value Per Share, Total Debt, Net Debt, Total Revenues, Operating Income, Revenue Per Share, Basic EPS, Normalized Basic EPS, Dividend Per Share, EBITA, EBIT, Normalized Net Income, Net Income, Cash from Operations, Cash from Investing, Cash from Financing, Net Change in Cash, Levered Free Cash Flow, Unlevered Free Cash Flow, Free Cash Flow / Share, Price]
Index: []

[0 rows x 29 columns]
Column: Time
Empty DataFrame
Columns: [Company, Time, Total Assets, Total Liabilities, Total Equity, Total Shares Out. on Filing Date, Book Value / Share, Tangible Book Value, Tangible Book Value Per Share, Total Debt, Net Debt, Total Revenues, Operating Income, Revenue Per Share, Basic EPS, Normalized Basic EPS, Dividend Per Share, EBITA, EBIT, Normalized Net Income, Net Income, Cash from Operations, Cash from I

In [7]:
# Shuffling the data to create random arrangement of data instances
data = data.sample(frac=1).reset_index(drop=True)
data

,Company,Time,Total Assets,Total Liabilities,Total Equity,Total Shares Out. on Filing Date,Book Value / Share,Tangible Book Value,Tangible Book Value Per Share,Total Debt,...,Normalized Net Income,Net Income,Cash from Operations,Cash from Investing,Cash from Financing,Net Change in Cash,Levered Free Cash Flow,Unlevered Free Cash Flow,Free Cash Flow / Share,Price
0,DVA,2019-12-01,17311.4,13811.8,3499.6,125.6,16.95,-4789.9,-38.06,11175.9,...,635.8,811.0,2496.2,2995.3,-4696.5,793.3,5199.3,5471.3,11.29,75.03
1,CVS,2012-12-01,66221.0,28568.0,37653.0,1231.2,30.59,1505.0,1.22,9828.0,...,4160.1,3864.0,6671.0,-1849.0,-4860.0,-38.0,4710.6,5061.3,3.65,48.35
2,NEM,2014-12-01,24916.0,11827.0,13089.0,498.9,20.60,10060.0,20.17,6646.0,...,777.6,508,1438.0,-507.0,-65.0,919,980.5,1186.8,0.80,18.44
3,TTWO,2019-03-01,4243.1,2202.5,2040.6,106.5,19.23,982.3,9.26,0.0,...,147,333.8,843.5,-223.6,-463.7,145.6,516.8,521.7,6.86,94.37
4,ADS,2018-12-01,30387.7,28055.6,2332.1,53.0,43.67,1072.4,20.08,25170.8,...,759.4,963.1,2754.9,-1872.0,-1217.9,-347.0,418.2,468.6,46.54,150.08
5,MU,2011-09-01,14752.0,4900.0,9852.0,987.6,8.61,8056.0,8.18,2001.0,...,228.3,167,2484.0,-2042.0,-1195.0,-753.0,126.6,147.1,-0.07,5.04
6,ALK,2015-12-01,6530.0,4119.0,2411.0,124.7,19.26,2411.0,19.26,701,...,840.6,848,1584.0,-930.0,-688.0,-34.0,557.3,562.3,5.87,80.51
7,KMX,2019-02-01,18717.9,15360.8,3357.0,166.2,20.04,3357.0,20.04,14172.9,...,695.5,842.4,163,-308.5,186,40.5,606.6,653.9,-0.81,62.10
8,MRWSY,2011-01-01,14493.5,5907.3,8586.1,2644.6,3.25,8294.7,3.14,1666.5,...,866.3,1001.2,1422.6,-922.0,-527.5,-26.9,574.3,608.9,0.18,21.14
9,ETR,2018-12-01,48275.1,39211.4,9063.7,189.6,46.78,8467.1,44.79,18132.6,...,302,848.7,2385.2,-4106.0,1420.4,-300.3,-2047.7,-1558.7,-9.68,86.07


In [8]:
#data.to_csv(r'C:\Users\conor\Documents\MScAIProject\MSc_AI_Project\Data\Testing\RawFilings_RawPrice\MergedStatements\HyperparameterTuning\MF_data.csv', index = False)